In [ ]:
# Define functions to connect to Google and change directories
def connectDrive():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

def changeDirectory(path):
    import os
    original_path = os.getcwd()
    os.chdir(path)
    new_path = os.getcwd()
    print("Original path: ",original_path)
    print("New path: ",new_path)

# Connect to Google Drive
connectDrive()

# Change path
changeDirectory("/content/drive/My Drive/github/find_p/")

In [ ]:
# !pip install vectorbt
# !pip install pandas_ta
# !pip install yfinance
# !pip install plotly
# !pip install numba
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4
# !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
# !chmod +x /usr/local/bin/orca

In [1]:
import numpy as np
import math
import vectorbt as vbt
import pandas as pd
from hyperopt import fmin, tpe, hp
from hyperopt.pyll import scope
from datetime import datetime
from pdta_vt_utils import dl, dtmask

# Portfolio Setting

In [2]:
cheight, cwidth = 500, 1000 # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("dark") # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D" # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 1000
vbt.settings.portfolio["fees"] = 0.0025 # 0.25%
vbt.settings.portfolio["slippage"] = 0.0025 # 0.25%
# vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False

pf_settings = pd.DataFrame(vbt.settings.portfolio.items(), columns=["Option", "Value"])
pf_settings.set_index("Option", inplace=True)

print(f"Portfolio Settings [Initial]")
pf_settings

Portfolio Settings [Initial]


,Value
Option,
call_seq,default
init_cash,1000
size,inf
size_type,amount
fees,0.0025
fixed_fees,0.0
slippage,0.0025
reject_prob,0.0
min_size,0.0


# Data Collection

In [3]:
########################## You can change this ##################
# You can add any asset to the list, just add to the list the ticker you want to test
# you can also remove any ticker of the list

asset_tickers = ["AAPL", "TSLA", "TWTR", "SPXL", "^GSPC", "SPY"]
##################################################################

print("="*100)
print("Tickers by index #")
print(f"    Assets: {', '.join([f'{k}: {v}' for k,v in enumerate(asset_tickers)])}")
print("-"*100)
assets = dl(asset_tickers, lc_cols=True)

start_date = datetime(2010, 1, 1) # Adjust as needed
start_date = pd.to_datetime(start_date).tz_localize('America/New_York')
end_date = datetime(2015, 1, 1)   # Adjust as needed
end_date = pd.to_datetime(end_date).tz_localize('America/New_York')

print("="*100)
print("Available Data:")
print("="*100)
print(f"Assets: {', '.join(assets.keys())}")
print("="*100)


Tickers by index #
    Assets: 0: AAPL, 1: TSLA, 2: TWTR, 3: SPXL, 4: ^GSPC, 5: SPY
Selected Benchmark | Asset: SPXL
[i] Downloading: AAPL, TSLA, TWTR, SPXL, ^GSPC, SPY
[+] AAPL(10632, 7) Monday February 13, 2023, NYSE: 15:08:28
[+] TSLA(3179, 7) Monday February 13, 2023, NYSE: 15:08:29
[+] TWTR(2259, 7) Monday February 13, 2023, NYSE: 15:08:29
[+] SPXL(3592, 8) Monday February 13, 2023, NYSE: 15:08:29
[+] ^GSPC(23894, 7) Monday February 13, 2023, NYSE: 15:08:30
[+] SPY(7565, 8) Monday February 13, 2023, NYSE: 15:08:30
[*] Download Complete

Available Data:
Assets: AAPL, TSLA, TWTR, SPXL, ^GSPC, SPY
Analysis of:  SPXL from 2010-01-01 00:00:00-05:00 to 2015-01-01 00:00:00-05:00


In [ ]:
########### You need to set this #############################
# Numero del indice que se quiere ver e.j. spy i=5
asset_ind = 5
##############################################################
asset_name = asset_tickers[3] # Change index for different symbol
print("="*100)
print(f"Selected Benchmark | Asset: {asset_name}")
print("="*100)

assetdf = assets[asset_name]

# Set True if you want to constrain Data between start_date & end_date
common_range = True
crs = ''
if common_range:
    crs = f" from {start_date} to {end_date}"
    assetdf = dtmask(assetdf, start_date, end_date)

# Update DataFrame names
assetdf.name = asset_name
print("="*100)
print(f"Analysis of:  {assetdf.name}{crs}")
print("="*100)

In [4]:
assetdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2010-01-04 00:00:00-05:00,4.100748,4.207281,4.099988,4.200433,28238400,0.0,0.0,0.0
2010-01-05 00:00:00-05:00,4.195868,4.242286,4.147168,4.240003,33206400,0.0,0.0,0.0
2010-01-06 00:00:00-05:00,4.227827,4.278810,4.217935,4.251416,44194800,0.0,0.0,0.0
2010-01-07 00:00:00-05:00,4.229349,4.315336,4.188258,4.301639,43773600,0.0,0.0,0.0
2010-01-08 00:00:00-05:00,4.264352,4.352622,4.240001,4.347294,39685200,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2014-12-24 00:00:00-05:00,21.372379,21.452074,21.266901,21.290340,1291600,0.0,0.0,0.0
2014-12-26 00:00:00-05:00,21.445042,21.606777,21.421602,21.480202,2223200,0.0,0.0,0.0
2014-12-29 00:00:00-05:00,21.400506,21.634904,21.391130,21.559896,2677600,0.0,0.0,0.0


# Indicator Setting Use this section for custome indicators

In [5]:
# def wavetrend(high, low, close, n1, n2):
#     ap = (high + low + close) / 3.0  # HLC3
#     ap = pd.DataFrame(ap)
#     esa = ap.ewm(span=n1, adjust=False).mean()  # EMA
#     d = (abs(ap - esa)).ewm(span=n1, adjust=False).mean()
#     ci = (ap - esa) / (0.015 * d)
#     wt1 = ci.ewm(span=n2, adjust=False).mean()
#     wt2 = wt1.rolling(4).mean()
#     return wt1, wt2
#
#
# Wavetrend = vbt.IndicatorFactory(
#     class_name='Wavetrend',
#     input_names=['high', 'low', 'close'],
#     param_names=['n1', 'n2'],
#     output_names=['wt1', 'wt2'], ).from_apply_func(wavetrend)
#


# Optimization

In [6]:
###### NOTE ##############################
# For future work:
# You can add the below and above to the optimization space
##########################################
below = 30,
above = 70


def optimize(params):
    # Parameters to optimize #
    below, above, window, sl_stop = params
    #Indicator
    rsi = vbt.RSI.run(assetdf.Close, window = window)

    #strategy
    entries = rsi.rsi_crossed_below(below)
    exits = rsi.rsi_crossed_above(above)

    #portfolio
    pf = vbt.Portfolio.from_signals(close=assetdf['Close'], entries=entries, exits=exits, sl_stop=sl_stop, direction="longonly")

    print(f"total profit {pf.total_profit()} with parameters: {below}, {above}, {window}, {sl_stop}")
    return np.array([pf.total_profit()])


def optimize_all(params):
    # optimization loss function (or objective function)
    res = optimize(params)
    return sum(res) * -1

# Search Space
sspace = [hp.uniform('below', 20, 60),
          hp.uniform('above', 70, 110),
          scope.int(hp.uniform('window', 7, 20)),
          hp.uniform('sl_stop', 0.01, 0.1)]


best = fmin(optimize_all,
            space=sspace,
            algo=tpe.suggest,
            max_evals=100)

print(best)

total profit 5.335220062048052 with parameters: 47.191353442351065, 87.62392977302024, 11, 0.04789061476397058
total profit 1012.3900244312314 with parameters: 28.927337140400265, 73.25596446460285, 13, 0.07380792540195359
total profit 1101.2759380202092 with parameters: 31.674139957905826, 73.92063207098543, 19, 0.05330585313538565
total profit 924.3250273828311 with parameters: 37.52128864346314, 86.74350213550221, 19, 0.036709619454525694
total profit -271.36808770108496 with parameters: 40.412093118060284, 85.68559125271906, 11, 0.04156120565565702
total profit -209.96068149103417 with parameters: 42.76121976728499, 83.3223561078895, 9, 0.06994390160045005
total profit -187.26369308629182 with parameters: 59.910951325598624, 70.2310281684813, 19, 0.08596941647842858
total profit -437.8396629197855 with parameters: 40.20343732635101, 78.81132662004299, 7, 0.07717119280406433
total profit 920.1499465690033 with parameters: 40.503475967034774, 89.6788897622362, 14, 0.01414295996666955

In [7]:
### Use the results from the optimization to prepare an optimal portfolio
# Just copy/paste the values or aproximate then to an integer
# You can also set the default parameters and test the particular case
########################################
above = 95 #math.ceil(best['k_window'])
below = 24 #math.ceil(best['d_window'])
window = 14 #math.ceil(best['d_ewm'])
sl_stop = best['sl_stop']

In [8]:
rsi = vbt.RSI.run(assetdf.Close, window = window)

entries = rsi.rsi_crossed_below(below)
exits = rsi.rsi_crossed_above(above)

pf = vbt.Portfolio.from_signals(close=assetdf['Close'], entries=entries, exits=exits, sl_stop=sl_stop, direction="longonly")

pf.stats()

Start                         2010-01-04 00:00:00-05:00
End                           2014-12-31 00:00:00-05:00
Period                               1258 days 00:00:00
Start Value                                      1000.0
End Value                                   7969.921923
Total Return [%]                             696.992192
Benchmark Return [%]                         389.505873
Max Gross Exposure [%]                            100.0
Total Fees Paid                               81.937243
Max Drawdown [%]                              53.809899
Max Drawdown Duration                 346 days 00:00:00
Total Trades                                          4
Total Closed Trades                                   3
Total Open Trades                                     1
Open Trade PnL                               899.694207
Win Rate [%]                                  66.666667
Best Trade [%]                               486.093337
Worst Trade [%]                               -6

In [9]:
pf.plot().show_png()

AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '7a524065-199f-4167-96d7-69d07b4bc41e',
              'x': array([datetime.datetime(2010, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2014, 12, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2014, 12, 31, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([ 4.20043278,  4.24000263,  4.25141573, ..., 21.55989647, 21.21298599,
                          20.56136513]),
              'yaxis': 'y'},
             {'customdata': array([[  0.        , 293.9096453 ,   2.49376559],
                                   [  2.        , 430.98666218,   3.21091391],
                                   [  4.        , 455.06377391,  18.7800297 ],
                                   [  6.        , 387.61638022,  17.63149056]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
              'legendgroup': '1',
              'marker': {'color': '#37B13F',
                         'line': {'color': 'rgb(38,123,44)', 'width': 1},
                         'size': 8,
                         'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy',
              'type': 'scatter',
              'uid': '47d4ffc3-aa2a-40b4-b580-3fbd94f08c3c',
              'x': array([datetime.datetime(2010, 2, 8, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2010, 7, 2, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2014, 8, 7, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2014, 12, 16, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([ 3.3939214 ,  2.98005873, 16.50760248, 18.19478377]),
              'yaxis': 'y'},
             {'customdata': array([[  1.        , 293.9096453 ,   3.22700872],
                                   [  3.        , 430.98666218,  18.87416519],
                                   [  5.        , 455.06377391,  17.71986896]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
              'legendgroup': '2',
              'marker': {'color': '#EA4335',
                         'line': {'color': 'rgb(181,31,18)', 'width': 1},
                         'size': 8,
                         'symbol': 'triangle-down'},
              'mode': 'markers',
              'name': 'Sell',
              'type': 'scatter',
              'uid': 'b7e6bafe-7834-4934-9732-655964ba69f2',
              'x': array([datetime.datetime(2010, 3, 15, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2014, 6, 10, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2014, 10, 13, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)],
                         dtype=object),
              'xaxis': 'x',
              'y': array(

In [ ]:
rsi.plot().show_png()